In [66]:
import os
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc

In [67]:
#Reading Training and Testing data into dataframes
df_train = pd.read_csv('/Users/priyankasharma/Desktop/training_v2.csv')
#print(df_train.head())
#df_train.describe()

df_test = pd.read_csv('/Users/priyankasharma/Desktop/unlabeled.csv')
#print(df_test.head())
#df_test.describe()

In [68]:
#df_test = df_test.loc[:,['encounter_id','hospital_death','apache_4a_hospital_death_prob', 'd1_lactate_max', 'd1_lactate_min', 'd1_sysbp_min' ,'d1_spo2_min', 
#                       'gcs_motor_apache', 'd1_mbp_min,d1_temp_min', 'd1_temp_max', 'temp_apache', 'pre_icu_los_days', 
#                       'heart_rate_apache', 'bmi', 'map_apache', 'age', 'd1_diasbp_min', 'd1_sysbp_max', 'd1_glucose_min',
#                       'd1_platelets_min', 'd1_bun_max', 'd1_bun_min', 'd1_mbp_max', 'd1_resprate_min', 'ph_apache', 
#                       'd1_platelets_max', 'd1_wbc_min', 'd1_wbc_max', 'd1_creatinine_max', 'd1_arterial_pco2_min',
#                       'd1_arterial_ph_max']]
#df_train =  df_train.loc[:,['encounter_id','hospital_death','apache_4a_hospital_death_prob', 'd1_lactate_max', 'd1_lactate_min', 'd1_sysbp_min' ,'d1_spo2_min', 
#                       'gcs_motor_apache', 'd1_mbp_min,d1_temp_min', 'd1_temp_max', 'temp_apache', 'pre_icu_los_days', 
#                       'heart_rate_apache', 'bmi', 'map_apache', 'age', 'd1_diasbp_min', 'd1_sysbp_max', 'd1_glucose_min',
#                      'd1_platelets_min', 'd1_bun_max', 'd1_bun_min', 'd1_mbp_max', 'd1_resprate_min', 'ph_apache', 
#                      'd1_platelets_max', 'd1_wbc_min', 'd1_wbc_max', 'd1_creatinine_max', 'd1_arterial_pco2_min',
#                       'd1_arterial_ph_max']]


df_test = df_test.loc[:,['encounter_id','hospital_death','apache_4a_hospital_death_prob',
'd1_bun_max',
'd1_sysbp_min',
'ventilated_apache',
'd1_spo2_min',
'd1_resprate_max',
'd1_lactate_min',
'd1_lactate_max',
'd1_resprate_min',
'h1_resprate_min',
'd1_hco3_min',
'd1_arterial_po2_min',
'bun_apache',
'd1_arterial_ph_max',
'd1_sysbp_max',
'd1_temp_min',
'creatinine_apache',
'apache_post_operative',
'd1_temp_max',
'd1_bun_min',
'gcs_motor_apache',
'wbc_apache',
'd1_platelets_min',
'urineoutput_apache',
'd1_mbp_min',
'd1_spo2_max',
'age',
'd1_hco3_max',
'd1_arterial_pco2_min',
'd1_inr_max',
'icu_admit_source',
'resprate_apache',
'd1_wbc_min',
'heart_rate_apache',
'bilirubin_apache',
'd1_diasbp_min',
'd1_arterial_po2_max',
'd1_sodium_min',
'hospital_admit_source',
'd1_sodium_max',
'd1_glucose_min',
'apache_2_diagnosis',
'd1_mbp_max',
'diabetes_mellitus',
'd1_platelets_max',
'bmi',
'pre_icu_los_days',
'solid_tumor_with_metastasis',
'fio2_apache',
'apache_3j_bodysystem',
'temp_apache',
'd1_potassium_min',
'map_apache',
'icu_type',
'd1_arterial_pco2_max',
'hematocrit_apache',
'd1_hemaglobin_max',
'd1_calcium_min',
'd1_hemaglobin_min',
'd1_glucose_max',
'albumin_apache',
'd1_creatinine_min',
'apache_2_bodysystem',
'pao2_apache',
'd1_creatinine_max',
'd1_diasbp_max',
'd1_wbc_max',
'd1_potassium_max',
'sodium_apache',
'd1_calcium_max',
'icu_stay_type',
'ph_apache',
'h1_resprate_max',]]
# 'aids',   
# 'cirrhosis',
# 'hepatic_failure',
# 'immunosuppression',
# 'leukemia',
# 'lymphoma',
# 'solid_tumor_with_metastasis' ]]

df_train = df_train.loc[:,['encounter_id','hospital_death','apache_4a_hospital_death_prob',
'd1_bun_max',
'd1_sysbp_min',
'ventilated_apache',
'd1_spo2_min',
'd1_resprate_max',
'd1_lactate_min',
'd1_lactate_max',
'd1_resprate_min',
'h1_resprate_min',
'd1_hco3_min',
'd1_arterial_po2_min',
'bun_apache',
'd1_arterial_ph_max',
'd1_sysbp_max',
'd1_temp_min',
'creatinine_apache',
'apache_post_operative',
'd1_temp_max',
'd1_bun_min',
'gcs_motor_apache',
'wbc_apache',
'd1_platelets_min',
'urineoutput_apache',
'd1_mbp_min',
'd1_spo2_max',
'age',
'd1_hco3_max',
'd1_arterial_pco2_min',
'd1_inr_max',
'icu_admit_source',
'resprate_apache',
'd1_wbc_min',
'heart_rate_apache',
'bilirubin_apache',
'd1_diasbp_min',
'd1_arterial_po2_max',
'd1_sodium_min',
'hospital_admit_source',
'd1_sodium_max',
'd1_glucose_min',
'apache_2_diagnosis',
'd1_mbp_max',
'diabetes_mellitus',
'd1_platelets_max',
'bmi',
'pre_icu_los_days',
'solid_tumor_with_metastasis',
'fio2_apache',
'apache_3j_bodysystem',
'temp_apache',
'd1_potassium_min',
'map_apache',
'icu_type',
'd1_arterial_pco2_max',
'hematocrit_apache',
'd1_hemaglobin_max',
'd1_calcium_min',
'd1_hemaglobin_min',
'd1_glucose_max',
'albumin_apache',
'd1_creatinine_min',
'apache_2_bodysystem',
'pao2_apache',
'd1_creatinine_max',
'd1_diasbp_max',
'd1_wbc_max',
'd1_potassium_max',
'sodium_apache',
'd1_calcium_max',
'icu_stay_type',
'ph_apache',
'h1_resprate_max',]]
# 'aids',   
# 'cirrhosis',
# 'hepatic_failure',
# 'immunosuppression',
# 'leukemia',
# 'lymphoma',
# 'solid_tumor_with_metastasis' ]]

df_train.describe()

,encounter_id,hospital_death,apache_4a_hospital_death_prob,d1_bun_max,d1_sysbp_min,ventilated_apache,d1_spo2_min,d1_resprate_max,d1_lactate_min,d1_lactate_max,...,d1_creatinine_min,pao2_apache,d1_creatinine_max,d1_diasbp_max,d1_wbc_max,d1_potassium_max,sodium_apache,d1_calcium_max,ph_apache,h1_resprate_max
count,91713.000000,91713.000000,83766.000000,81199.000000,91554.00000,90998.000000,91380.000000,91328.000000,23317.000000,23317.000000,...,81544.000000,20845.000000,81544.000000,91548.000000,78539.000000,82128.000000,73113.000000,78644.000000,20845.000000,87356.000000
mean,65606.079280,0.086302,0.086787,25.685745,96.92387,0.325721,90.454826,28.882774,2.125128,2.927383,...,1.366947,131.148467,1.488650,88.491873,12.476302,4.251594,137.966373,8.378881,7.353895,22.633614
std,37795.088538,0.280811,0.247569,20.468943,20.67793,0.468646,10.030069,10.701973,2.111479,3.075982,...,1.333855,83.607292,1.506957,19.798379,6.796187,0.667355,5.279418,0.735581,0.097755,7.515043
min,1.000000,0.000000,-1.000000,4.000000,41.00000,0.000000,0.000000,14.000000,0.400000,0.400000,...,0.300000,31.000000,0.340000,46.000000,1.200000,2.800000,117.000000,6.200000,6.960540,10.000000
25%,32852.000000,0.000000,0.020000,13.000000,83.00000,0.000000,89.000000,22.000000,1.000000,1.200000,...,0.710000,77.500000,0.760000,75.000000,8.000000,3.800000,135.000000,7.900000,7.308000,18.000000
50%,65665.000000,0.000000,0.050000,19.000000,96.00000,0.000000,92.000000,26.000000,1.500000,1.900000,...,0.950000,103.500000,1.000000,86.000000,11.000000,4.200000,138.000000,8.400000,7.360000,21.000000
75%,98342.000000,0.000000,0.130000,31.000000,110.00000,1.000000,95.000000,32.000000,2.300000,3.300000,...,1.400000,153.000000,1.500000,99.000000,15.200000,4.600000,141.000000,8.800000,7.419000,26.000000
max,131051.000000,1.000000,0.990000,126.000000,160.00000,1.000000,100.000000,92.000000,15.100000,19.800000,...,9.937900,498.000000,11.110000,165.000000,46.080000,7.000000,158.000000,10.800000,7.590000,59.000000


In [69]:
Train = df_train.copy(deep=True)
Test = df_test.copy(deep=True)

In [70]:
# replacing nan
Train['hospital_admit_source'] = Train['hospital_admit_source'].replace(np.NaN,'Unknown')
Train['icu_admit_source'] = Train['icu_admit_source'].replace(np.NaN,'Unknown')
Train['icu_type'] = Train['icu_type'].replace(np.NaN,'Unknown') # no Nan found
Train['icu_stay_type'] = Train['icu_stay_type'].replace(np.NaN,'Unknown')  # no Nan found
Train['apache_3j_bodysystem'] = Train['apache_3j_bodysystem'].replace(np.NaN,'Unknown')  # no Nan found
Train['apache_2_bodysystem'] = Train['apache_2_bodysystem'].replace(np.NaN,'Unknown') # no Nan found


Test['hospital_admit_source'] = Test['hospital_admit_source'].replace(np.NaN,'Unknown')
Test['icu_admit_source'] = Test['icu_admit_source'].replace(np.NaN,'Unknown')
Test['icu_stay_type'] = Test['icu_stay_type'].replace(np.NaN,'Unknown')
Test['icu_type'] = Test['icu_type'].replace(np.NaN,'Unknown')
Test['apache_3j_bodysystem'] = Test['apache_3j_bodysystem'].replace(np.NaN,'Unknown')
Test['apache_2_bodysystem'] = Test['apache_2_bodysystem'].replace(np.NaN,'Unknown')

In [71]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
Train['hospital_admit_source'] = lbl.fit_transform(Train['hospital_admit_source'].astype(str))
Train['icu_admit_source'] = lbl.fit_transform(Train['icu_admit_source'].astype(str))
Train['icu_stay_type'] = lbl.fit_transform(Train['icu_stay_type'].astype(str))
Train['icu_type'] = lbl.fit_transform(Train['icu_type'].astype(str))
Train['apache_3j_bodysystem'] = lbl.fit_transform(Train['apache_3j_bodysystem'].astype(str))
Train['apache_2_bodysystem'] = lbl.fit_transform(Train['apache_2_bodysystem'].astype(str))

Test['hospital_admit_source'] = lbl.fit_transform(Test['hospital_admit_source'].astype(str))
Test['icu_admit_source'] = lbl.fit_transform(Test['icu_admit_source'].astype(str))
Test['icu_stay_type'] = lbl.fit_transform(Test['icu_stay_type'].astype(str))
Test['icu_type'] = lbl.fit_transform(Test['icu_type'].astype(str))
Test['apache_3j_bodysystem'] = lbl.fit_transform(Test['apache_3j_bodysystem'].astype(str))
Test['apache_2_bodysystem'] = lbl.fit_transform(Test['apache_2_bodysystem'].astype(str))

In [72]:
Train["d1_temp_max"].fillna(37, inplace = True) 
Train["d1_temp_min"].fillna(37, inplace = True) 

Train["d1_sysbp_max"].fillna(120, inplace = True)
Train["d1_sysbp_min"].fillna(120, inplace = True)

Train["d1_diasbp_max"].fillna(80, inplace = True)
Train["d1_diasbp_min"].fillna(80, inplace = True)


Train["d1_resprate_max"].fillna(14, inplace = True)
Train["d1_resprate_min"].fillna(14, inplace = True)
Train["h1_resprate_max"].fillna(14, inplace = True)
Train["h1_resprate_min"].fillna(14, inplace = True)

Train["d1_hco3_max"].fillna(25.5, inplace = True)
Train["d1_hco3_min"].fillna(25.5, inplace = True)

Train["d1_glucose_max"].fillna(90, inplace = True)
Train["d1_glucose_min"].fillna(90, inplace = True)

Train["d1_lactate_max"].fillna(2.2, inplace = True)
Train["d1_lactate_min"].fillna(2.2, inplace = True)

Train["d1_potassium_max"].fillna(4.25, inplace = True)
Train["d1_potassium_min"].fillna(4.25, inplace = True)

Train["d1_bun_max"].fillna(13.5, inplace = True)
Train["d1_bun_min"].fillna(13.5, inplace = True)

Train["d1_calcium_max"].fillna(9.35, inplace = True)
Train["d1_calcium_min"].fillna(9.35, inplace = True)

Train["d1_creatinine_max"].fillna(0.9, inplace = True)
Train["d1_creatinine_min"].fillna(0.9, inplace = True)

Train["d1_hemaglobin_max"].fillna(13.75, inplace = True)
Train["d1_hemaglobin_min"].fillna(13.75, inplace = True)


Train["d1_wbc_max"].fillna(7.7, inplace = True)
Train["d1_wbc_min"].fillna(7.7, inplace = True)


Train["d1_platelets_max"].fillna(300, inplace = True)
Train["d1_platelets_min"].fillna(300, inplace = True)


In [73]:
Test["d1_temp_max"].fillna(37, inplace = True) 
Test["d1_temp_min"].fillna(37, inplace = True) 

Test["d1_sysbp_max"].fillna(120, inplace = True)
Test["d1_sysbp_min"].fillna(120, inplace = True)

Test["d1_diasbp_max"].fillna(80, inplace = True)
Test["d1_diasbp_min"].fillna(80, inplace = True)


Test["d1_resprate_max"].fillna(14, inplace = True)
Test["d1_resprate_min"].fillna(14, inplace = True)
Test["h1_resprate_max"].fillna(14, inplace = True)
Test["h1_resprate_min"].fillna(14, inplace = True)

Test["d1_hco3_max"].fillna(25.5, inplace = True)
Test["d1_hco3_min"].fillna(25.5, inplace = True)

Test["d1_glucose_max"].fillna(90, inplace = True)
Test["d1_glucose_min"].fillna(90, inplace = True)

Test["d1_lactate_max"].fillna(2.2, inplace = True)
Test["d1_lactate_min"].fillna(2.2, inplace = True)

Test["d1_potassium_max"].fillna(4.25, inplace = True)
Test["d1_potassium_min"].fillna(4.25, inplace = True)

Test["d1_bun_max"].fillna(13.5, inplace = True)
Test["d1_bun_min"].fillna(13.5, inplace = True)

Test["d1_calcium_max"].fillna(9.35, inplace = True)
Test["d1_calcium_min"].fillna(9.35, inplace = True)

Test["d1_creatinine_max"].fillna(0.9, inplace = True)
Test["d1_creatinine_min"].fillna(0.9, inplace = True)

Test["d1_hemaglobin_max"].fillna(13.75, inplace = True)
Test["d1_hemaglobin_min"].fillna(13.75, inplace = True)


Test["d1_wbc_max"].fillna(7.7, inplace = True)
Test["d1_wbc_min"].fillna(7.7, inplace = True)


Test["d1_platelets_max"].fillna(300, inplace = True)
Test["d1_platelets_min"].fillna(300, inplace = True)


In [74]:
X_col_dropped = ['hospital_death']
X_train_val = Train.drop(labels=X_col_dropped, axis=1)
#print(X.dtypes)
X_col_names = X_train_val.columns.values
X_col_names

array(['encounter_id', 'apache_4a_hospital_death_prob', 'd1_bun_max',
       'd1_sysbp_min', 'ventilated_apache', 'd1_spo2_min',
       'd1_resprate_max', 'd1_lactate_min', 'd1_lactate_max',
       'd1_resprate_min', 'h1_resprate_min', 'd1_hco3_min',
       'd1_arterial_po2_min', 'bun_apache', 'd1_arterial_ph_max',
       'd1_sysbp_max', 'd1_temp_min', 'creatinine_apache',
       'apache_post_operative', 'd1_temp_max', 'd1_bun_min',
       'gcs_motor_apache', 'wbc_apache', 'd1_platelets_min',
       'urineoutput_apache', 'd1_mbp_min', 'd1_spo2_max', 'age',
       'd1_hco3_max', 'd1_arterial_pco2_min', 'd1_inr_max',
       'icu_admit_source', 'resprate_apache', 'd1_wbc_min',
       'heart_rate_apache', 'bilirubin_apache', 'd1_diasbp_min',
       'd1_arterial_po2_max', 'd1_sodium_min', 'hospital_admit_source',
       'd1_sodium_max', 'd1_glucose_min', 'apache_2_diagnosis',
       'd1_mbp_max', 'diabetes_mellitus', 'd1_platelets_max', 'bmi',
       'pre_icu_los_days', 'solid_tumor_with_me

In [75]:
y_train_val = Train.loc[:,'hospital_death']
print(y_train_val.unique())
y_train_val

[0 1]


0        0
1        0
2        0
3        0
4        0
        ..
91708    0
91709    0
91710    0
91711    0
91712    0
Name: hospital_death, Length: 91713, dtype: int64

In [76]:
X_test_col_dropped = ['hospital_death']
X_test = Test.drop(labels=X_test_col_dropped, axis=1)
print(X_test.columns.values)

['encounter_id' 'apache_4a_hospital_death_prob' 'd1_bun_max'
 'd1_sysbp_min' 'ventilated_apache' 'd1_spo2_min' 'd1_resprate_max'
 'd1_lactate_min' 'd1_lactate_max' 'd1_resprate_min' 'h1_resprate_min'
 'd1_hco3_min' 'd1_arterial_po2_min' 'bun_apache' 'd1_arterial_ph_max'
 'd1_sysbp_max' 'd1_temp_min' 'creatinine_apache' 'apache_post_operative'
 'd1_temp_max' 'd1_bun_min' 'gcs_motor_apache' 'wbc_apache'
 'd1_platelets_min' 'urineoutput_apache' 'd1_mbp_min' 'd1_spo2_max' 'age'
 'd1_hco3_max' 'd1_arterial_pco2_min' 'd1_inr_max' 'icu_admit_source'
 'resprate_apache' 'd1_wbc_min' 'heart_rate_apache' 'bilirubin_apache'
 'd1_diasbp_min' 'd1_arterial_po2_max' 'd1_sodium_min'
 'hospital_admit_source' 'd1_sodium_max' 'd1_glucose_min'
 'apache_2_diagnosis' 'd1_mbp_max' 'diabetes_mellitus' 'd1_platelets_max'
 'bmi' 'pre_icu_los_days' 'solid_tumor_with_metastasis' 'fio2_apache'
 'apache_3j_bodysystem' 'temp_apache' 'd1_potassium_min' 'map_apache'
 'icu_type' 'd1_arterial_pco2_max' 'hematocrit_apache

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
median_features = ['d1_spo2_min','d1_arterial_po2_min','d1_arterial_ph_max','d1_mbp_min','d1_spo2_max','d1_arterial_pco2_min',
                  'd1_inr_max','d1_arterial_po2_max','d1_sodium_max','d1_sodium_min','d1_mbp_max','d1_arterial_pco2_max', 'bun_apache',
                  'creatinine_apache','gcs_motor_apache','urineoutput_apache','wbc_apache','age', 'bmi','resprate_apache',
                  'heart_rate_apache','bilirubin_apache','sodium_apache','ph_apache','pao2_apache','hematocrit_apache',
                  'map_apache','temp_apache','albumin_apache','fio2_apache','apache_4a_hospital_death_prob','apache_2_diagnosis',]

median_transformer = Pipeline(steps=
                              [
                                ('numeric', SimpleImputer(missing_values=np.nan, strategy='median'))
                              ])

mode_features = ['ventilated_apache','apache_post_operative','apache_3j_bodysystem','apache_3j_bodysystem','diabetes_mellitus']

mode_transformer = Pipeline(steps=
                              [
                                ('categorical', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
                              ])

preprocessor = ColumnTransformer(
    remainder='passthrough',
    transformers=[('numeric', median_transformer, median_features),
                  ('categorical', mode_transformer, mode_features)],
    n_jobs=-1
)

In [78]:
X_train_val_imp = preprocessor.fit_transform(X_train_val)
X_test_imp = preprocessor.transform(X_test)
print(X_train_val_imp.shape)
print(X_test_imp.shape)
print(type(X_train_val_imp),type(X_test_imp))

(91713, 75)
(39308, 75)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [79]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val_imp, y_train_val, test_size=0.2, random_state=1, stratify=y_train_val)
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))

73370 73370
18343 18343


In [80]:
xgb =   XGBClassifier (scale_pos_weight = 1, max_depth = 15, subsample = .75, eta = .03, n_estimators = 3000) 
fitted_model = xgb.fit(X_train, y_train)
print(fitted_model)
y_pred = fitted_model.predict(X_val)
pTot = accuracy_score(y_true=y_val, y_pred=y_pred)
pTot

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.03, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)


0.9333260644387504

In [81]:
cm = confusion_matrix(y_val, y_pred)
print(cm)
report = classification_report(y_val, y_pred)

[[16579   181]
 [ 1042   541]]


In [82]:
probas_ = fitted_model.predict_proba(X_val)
fpr, tpr, thresholds = roc_curve(y_val, probas_[:, 1])
roc_auc = auc(fpr, tpr)
print ("thresholds", thresholds)
print ("probas_", probas_)
print ("AUC using predict_proba", roc_auc)

thresholds [1.9999999e+00 9.9999988e-01 9.9999607e-01 ... 8.3662698e-08 8.3091948e-08
 9.2742587e-09]
probas_ [[9.98499215e-01 1.50077348e-03]
 [9.99985814e-01 1.42156905e-05]
 [9.21943069e-01 7.80569017e-02]
 ...
 [7.46096611e-01 2.53903389e-01]
 [9.99971151e-01 2.88667234e-05]
 [9.98634279e-01 1.36572763e-03]]
AUC using predict_proba 0.8979677419841182


In [83]:
fitted_model_train = xgb.fit(X_train_val_imp, y_train_val)
probas_test = fitted_model_train.predict_proba(X_test_imp)
probas_test.shape

(39308, 2)

In [84]:
probas_test

array([[9.9993306e-01, 6.6960638e-05],
       [9.9985635e-01, 1.4364164e-04],
       [9.9999344e-01, 6.5441304e-06],
       ...,
       [9.9961543e-01, 3.8456824e-04],
       [9.9973989e-01, 2.6009022e-04],
       [9.9587858e-01, 4.1214507e-03]], dtype=float32)

In [85]:
col_encounter = Test['encounter_id']
print(type(col_encounter))
print(col_encounter.dtype)
col_encounter

<class 'pandas.core.series.Series'>
int64


0             2
1             5
2             7
3             8
4            10
          ...  
39303    131035
39304    131037
39305    131039
39306    131041
39307    131050
Name: encounter_id, Length: 39308, dtype: int64

In [86]:
probas_test[:,1]

array([6.6960638e-05, 1.4364164e-04, 6.5441304e-06, ..., 3.8456824e-04,
       2.6009022e-04, 4.1214507e-03], dtype=float32)

In [87]:
probas_survival = probas_test[:,1]
probas_survival = pd.Series(probas_survival)
print(probas_survival.shape)
print(type(probas_survival))
print(probas_survival.dtype)
probas_survival

(39308,)
<class 'pandas.core.series.Series'>
float32


0        6.696064e-05
1        1.436416e-04
2        6.544130e-06
3        1.133790e-02
4        8.642240e-01
             ...     
39303    6.848469e-07
39304    1.660665e-05
39305    3.845682e-04
39306    2.600902e-04
39307    4.121451e-03
Length: 39308, dtype: float32

In [88]:
result_dict = { 'encounter_id': col_encounter, 'hospital_death': probas_survival }
result_df = pd.DataFrame(result_dict)
print(type(result_df))
print(result_df.dtypes)
result_df.head(5)

<class 'pandas.core.frame.DataFrame'>
encounter_id        int64
hospital_death    float32
dtype: object


,encounter_id,hospital_death
0,2,0.000067
1,5,0.000144
2,7,0.000007
3,8,0.011338
4,10,0.864224


In [89]:
result_df.to_csv('/Users/priyankasharma/Desktop/output_xgbA19.csv', index= False)